In [1]:
from google.colab import drive
import pandas as pd
import ast
from collections import defaultdict
from datetime import timedelta

In [2]:
file_path = 'final_emails.csv'
df = pd.read_csv(file_path)
df.head()

,From,Date,Subject,Body,Recipients,Domains,Label
0,lorna.brennan@enron.com,2000-11-01 02:19:00,Lay's Comments to FERC on Power Market,Note: The FERC meeting is today. See the link ...,"['jeff.nielsen@enron.com', 'laura.lantefield@e...",['intelligencepress.com'],0
1,john.allison@enron.com,2001-11-21 12:50:10,ERV Notification: (Enron Americas Position Re...,The report named: Enron Americas Position Repo...,"['frank.hayden@enron.com', 'louise.kitchen@enr...",['erv.corp.enron.com'],0
2,michelle.cash@enron.com,2001-10-26 13:15:28,Fw: RE:,Never mind.,['twanda.sweet@enron.com'],['blackberry.net'],0
3,lara.leibman@enron.com,2001-03-02 08:32:00,FTC Probe into Retail Sales,"Jeff, This is the info that I referenced on my...",['jeff.dasovich@enron.com'],['ftc.gov'],0
4,james.derrick@enron.com,2001-10-25 16:43:54,FW: <<Concur Expense Document>> - Lunch,FYI,['j.harris@enron.com'],['expensexms.enron.com'],0


In [4]:
phishing_df = df[df['Label'] == 1]
phishing_df.sample(5)

,From,Date,Subject,Body,Recipients,Domains,Label
3886,david.delainey@enron.com,2001-01-08 11:58:48,Monthly Financial Overview,"Hi team, I've compiled the latest financial ov...","['winston.goodbody@enron.com', 'jeff.huo@enron...",['enron-docs.com'],1
11566,david.delainey@enron.com,2000-11-08 15:41:05,Review Required: Updated Financial Projections,"Hi Kay and David, I've just uploaded the updat...","['kay.chapman@enron.com', 'david.oxley@enron.c...",['secure-docs-review.com'],1
5448,stephanie.sever@enron.com,2002-04-11 17:04:49,Monthly Budget Review,"Hello team, I've uploaded the latest budget re...","['david.ryan@enron.com', 'john.oh@enron.com', ...",['budget-report-review.com'],1
6274,stephanie.sever@enron.com,2002-04-04 13:29:41,Monthly Report Access,"Hi Team, I've uploaded the latest monthly repo...","['chris.benham@enron.com', 'jim.elston@enron.c...",['enron-files-review.com'],1
1268,david.delainey@enron.com,2000-12-04 14:27:17,Team Project Update,"Hello team, I've uploaded the latest version o...","['brandon.oliveira@enron.com', 'vladi.pimenov@...",['enronfiles-update.com'],1


In [5]:
df.count()

,0
From,13184
Date,13184
Subject,13184
Body,13184
Recipients,13184
Domains,13184
Label,13184


In [6]:
df['Date'] = pd.to_datetime(df['Date'], errors='raise')

In [7]:
print(df['Date'].dtype)

datetime64[ns]


In [8]:
print(type(df['Recipients'].iloc[0]))
print(type(df['Domains'].iloc[0]))

<class 'str'>
<class 'str'>


In [9]:
# Convert stringified lists to sets of lowercase, stripped values
df['Recipients'] = df['Recipients'].apply(
    lambda x: set(email.strip().lower() for email in ast.literal_eval(x)) if pd.notnull(x) else set()
)

df['Domains'] = df['Domains'].apply(
    lambda x: set(domain.strip().lower() for domain in ast.literal_eval(x)) if pd.notnull(x) else set()
)

In [10]:
print(type(df['Recipients'].iloc[0]))
print(type(df['Domains'].iloc[0]))

<class 'set'>
<class 'set'>


In [11]:
df.head()

,From,Date,Subject,Body,Recipients,Domains,Label
0,lorna.brennan@enron.com,2000-11-01 02:19:00,Lay's Comments to FERC on Power Market,Note: The FERC meeting is today. See the link ...,"{donna.martens@enron.com, christine.stokes@enr...",{intelligencepress.com},0
1,john.allison@enron.com,2001-11-21 12:50:10,ERV Notification: (Enron Americas Position Re...,The report named: Enron Americas Position Repo...,"{kenneth.thibodeaux@enron.com, david.port@enro...",{erv.corp.enron.com},0
2,michelle.cash@enron.com,2001-10-26 13:15:28,Fw: RE:,Never mind.,{twanda.sweet@enron.com},{blackberry.net},0
3,lara.leibman@enron.com,2001-03-02 08:32:00,FTC Probe into Retail Sales,"Jeff, This is the info that I referenced on my...",{jeff.dasovich@enron.com},{ftc.gov},0
4,james.derrick@enron.com,2001-10-25 16:43:54,FW: <<Concur Expense Document>> - Lunch,FYI,{j.harris@enron.com},{expensexms.enron.com},0


In [12]:
phishy_keywords = [
    "verify", "reset your password", "confirm your identity", "sign in", "unauthorized login",
    "your account", "update your account", "security alert", "click here", "login attempt",
    "secure message", "reactivate", "reset password", "confirm account", "your credentials",
    "important notice", "urgent", "immediate action", "unusual activity", "suspicious login",
    "account locked", "account suspended", "you must", "action required", "follow the link",
    "verify your email", "check the attachment", "shared document", "document has been shared",
    "view document", "dropbox", "onedrive", "sharepoint", "google drive", "view attachment",
    "encrypted message", "compliance notice", "security update", "new device", "you have received a message"
]

In [13]:
url = "http://s3-us-west-1.amazonaws.com/umbrella-static/top-1m.csv.zip"

df_umbrella = pd.read_csv(url, compression='zip', header=None, names=['Rank', 'Domain'])
domain_rank = dict(zip(df_umbrella['Domain'], df_umbrella['Rank']))

In [14]:
df_umbrella.head()

,Rank,Domain
0,1,google.com
1,2,microsoft.com
2,3,data.microsoft.com
3,4,events.data.microsoft.com
4,5,www.google.com


In [15]:
domain_history = defaultdict(set)
recipient_history = defaultdict(list)

In [16]:
df = df.sort_values('Date').reset_index(drop=True)

In [17]:
df.head()

,From,Date,Subject,Body,Recipients,Domains,Label
0,vince.kaminski@enron.com,2000-01-04 00:11:00,Re: Happy New Year!,"Frank, Happy New Year and New Millennium. Vinc...",{fhayden@kpmg.com},"{clicktomarket.com, garp.com, secure.garp.com}",0
1,drew.fossum@enron.com,2000-01-04 04:11:00,Re: More on El Paso Protest of ENA Purchased C...,"I think you have a conflict for that one, but ...",{susan.scott@enron.com},"{energyseminars.com, energyseminars.co, gasmar...",0
2,chris.dorland@enron.com,2000-01-04 05:25:00,Fwd: FW: Fw: Is there a theme?,---------------------- Forwarded by Chris Dorl...,{cddorlan@telusplanet.net},{hotmail.com},0
3,vince.kaminski@enron.com,2000-01-05 02:45:00,Greetings from Richard,---------------------- Forwarded by Vince J Ka...,{vkaminski@aol.com},"{ynot.com, preview.ynot.com, macromedia.com}",0
4,vince.kaminski@enron.com,2000-01-05 02:54:00,"Your card has been sent, or, if it's a delayed...",---------------------- Forwarded by Vince J Ka...,{vkaminski@aol.com},{preview.ynot.com},0


In [18]:
for i, row in df.iterrows():
    current_date = row['Date'].date()
    sender = row['From']
    recipients = row['Recipients']
    domains = row['Domains']

    recipient_history[sender].append((current_date, recipients))
    domain_history[current_date].update(domains)

In [19]:
# Look at a sample sender's history
print(recipient_history['mark.kostinec@enron.com'][:2])

# Look at domains seen on a specific day
some_date = df['Date'].dt.date.iloc[0]
print(domain_history[some_date])

[]
{'clicktomarket.com', 'energyseminars.com', 'energyseminars.co', 'gasmart.com', 'garp.com', 'nesanet.org', 'secure.garp.com', 'austingrp.com', 'hotmail.com', 'energy.com'}


In [20]:
def contains_phishy_keywords(text, keywords):
    text = text.lower()
    return int(any(keyword in text for keyword in keywords))

df['HasPhishyKeywords'] = df.apply(
    lambda row: contains_phishy_keywords(row['Subject'] + " " + row['Body'], phishy_keywords),
    axis=1
)

In [21]:
df.groupby('Label')['HasPhishyKeywords'].value_counts()

Label  HasPhishyKeywords
0      0                    12652
       1                      403
1      0                       99
       1                       30
Name: count, dtype: int64

In [22]:
df['NumRecipients'] = df['Recipients'].apply(len)

In [23]:
df['NumRecipients'].describe()

,NumRecipients
count,13184.000000
mean,9.541338
std,40.909869
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,501.000000


In [24]:
def get_global_url_rank(domains, domain_rank, default_rank=10_000_000):
    if not domains:
        return default_rank
    ranks = [domain_rank.get(domain, default_rank) for domain in domains]
    return min(ranks)

df['GlobalURLRank'] = df['Domains'].apply(lambda d: get_global_url_rank(d, domain_rank))

In [25]:
df['GlobalURLRank'].describe()

,GlobalURLRank
count,1.318400e+04
mean,6.955529e+06
std,4.503145e+06
min,1.000000e+00
25%,4.790400e+05
50%,1.000000e+07
75%,1.000000e+07
max,1.000000e+07


In [26]:
local_url_freqs = []

for i, row in df.iterrows():
    current_date = row['Date'].date()
    domains = row['Domains']
    count = 0

    for delta in range(1, 31):  # Look back 30 days (excluding today)
        prev_date = current_date - timedelta(days=delta)
        if domains & domain_history.get(prev_date, set()):
            count += 1

    local_url_freqs.append(count)

# Assign to DataFrame
df['LocalURLFreq'] = local_url_freqs


In [27]:
df['LocalURLFreq'].describe()

,LocalURLFreq
count,13184.000000
mean,6.286711
std,8.528645
min,0.000000
25%,0.000000
50%,2.000000
75%,10.000000
max,30.000000


In [28]:
df.groupby('Label')['LocalURLFreq'].describe()

,count,mean,std,min,25%,50%,75%,max
Label,,,,,,,,
0,13055.0,6.348372,8.547955,0.0,0.0,2.0,10.0,30.0
1,129.0,0.046512,0.211411,0.0,0.0,0.0,0.0,1.0


In [29]:
recipient_likelihoods = []

for i, row in df.iterrows():
    current_date = row['Date'].date()
    sender = row['From']
    recipients = row['Recipients']

    max_jaccard = 0.0

    for past_date, past_recipients in recipient_history[sender]:
        # Skip emails outside the 30-day window
        days_diff = (current_date - past_date).days
        if 0 < days_diff <= 30:
            union = recipients | past_recipients
            if union:  # avoid division by zero
                jaccard = len(recipients & past_recipients) / len(union)
                max_jaccard = max(max_jaccard, jaccard)

    recipient_likelihoods.append(max_jaccard)

# Assign to DataFrame
df['RecipientLikelihood'] = recipient_likelihoods

In [30]:
df.groupby('Label')['RecipientLikelihood'].describe()

,count,mean,std,min,25%,50%,75%,max
Label,,,,,,,,
0,13055.0,0.364806,0.455041,0.000,0.000000,0.000000,1.000000,1.0
1,129.0,0.330075,0.322723,0.002,0.147776,0.194743,0.212121,1.0


In [31]:
df_features = df[['Date','Label', 'HasPhishyKeywords', 'NumRecipients', 'GlobalURLRank', 'LocalURLFreq', 'RecipientLikelihood']]

In [41]:
df_features.sample(10)

,Date,Label,HasPhishyKeywords,NumRecipients,GlobalURLRank,LocalURLFreq,RecipientLikelihood
2904,2000-12-19 00:21:00,0,0,14,881256,0,0.071429
8601,2001-10-11 09:05:48,0,0,1,479040,13,0.000000
11060,2001-11-25 03:52:00,0,0,1,479040,30,1.000000
12896,2002-03-05 15:04:30,0,0,1,10000000,0,1.000000
1715,2000-10-24 07:24:00,0,0,1,10000000,1,0.142857
2523,2000-11-30 03:16:00,0,0,2,10000000,0,1.000000
12895,2002-03-05 14:20:02,0,0,1,10000000,2,0.000000
8335,2001-10-02 05:46:40,0,0,1,193,0,0.000000
1440,2000-10-02 06:25:00,0,0,1,10000000,8,0.000000
6692,2001-06-18 16:37:35,0,1,112,10000000,4,0.840708


In [43]:
df_features.to_csv("emails_features.csv", index=False)